In [ ]:
!pip install PySimpleGUI
!pip install pandas
!pip install selenium
!pip install beautifulsoup4

In [ ]:
#to enable chrome driver 
#based on your chrome browser version, install chrome driver
#click on help in your chrome browser, 
#check the version and install the supporting version

#various available versions: 2.38, 2.45.2, 2.45.3, 2.46, 78.0.3904.11, 78.0.3904.70, 
#79.0.3945.16, 79.0.3945.36, 80.0.3987.16, 81.0.4044.20, 
#81.0.4044.69, 83.0.4103.14, 83.0.4103.39, 84.0.4147.30, 
#85.0.4183.38, 85.0.4183.83, 85.0.4183.87, 86.0.4240.22, 
#87.0.4280.20, 87.0.4280.88, 88.0.4324.27, 88.0.4324.96, 
#89.0.4389.23, 90.0.4430.24, 91.0.4472.1

!pip install chromedriver-py==90.0.4430.24

In [1]:
import PySimpleGUI as sg
from chromedriver_py import binary_path
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import re

sg.theme("SystemDefault")

def scraping(url):
    driver = webdriver.Chrome(executable_path=binary_path)
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    x = soup.find("div",{"id":"filter-info-section"})
    value = x.get_text()
    
    r = ""
    
    for i in range(0,len(value)):
        if value[i] == "|":
            for i in range(i+1,len(value)):
                r = r+value[i]

    r = r.replace(',','')

    n = re.findall(r'\d+', r)

    last = (int(n[0])//10) + 1 

    if (int(n[0])%10) !=0:
        last = last+ 1 
    
    customers = []
    titles = []
    reviews = []
    ratings = []

    for page in range(1,last):
        if page == 1:
            driver.get(url)
            soup = BeautifulSoup(driver.page_source,'html.parser')
    
        else:
        
            l = ""
            x = soup.find('li',{'class':'a-last'})
            for link in x.findAll('a', attrs={'href':re.compile('/')}):
                l = link.get('href')
        
            url = "https://www.amazon.in" + l
            
            driver.get(url.format(page,page))
            soup = BeautifulSoup(driver.page_source,'html.parser')
        
        
        #retirving customer names
        names =soup.find_all('span',{'class':'a-profile-name'})

   
        for i in range(2,len(names)):
            customers.append(names[i].get_text())
        
        #review titles
        title = soup.find_all('a',{'data-hook':'review-title'})
        
        for i in range(0,len(title)):
            x = title[i].get_text()
            titles.append(x[1:len(x)-1])
        
        #review given by the customer
        review = soup.find_all("span",{"data-hook":"review-body"})

        for i in range(0,len(review)):
            x = review[i].get_text()
            reviews.append(x[4:len(x)-2])
        
        #rating given by the customer
        rating = soup.find_all('i',{'class':'review-rating'})

        for i in range(2,len(rating)):
            x = rating[i].get_text()
            ratings.append(x[0:3])
        
    driver.close()
    
    output["Customer Name"] = customers
    output["Review Title"] = titles
    output["Review"] = reviews
    output["Rating"] = ratings
    
    return r
    
output = pd.DataFrame(columns = ["Customer Name",'Review Title', 'Review', 'Rating'])

layout = [[sg.Text("Welcome to Amazon Web Scraping Application",justification = "center",size = (50,1),font=("Helvetica", 15))],
          [sg.Text("")],
          [sg.Text("NOTE: Please click on 'See all reviews' button of a particular product and paste the link here!",font=("Helvetica", 11))],
          [sg.Text("          "),sg.InputText(),sg.Button('Scrap Reviews')],
          [sg.Text("          "),sg.Text("",size = (50,1),key = "result1",font=("Helvetica", 10))],
          [sg.Text("")],
          [sg.Text("Please enter the file name to be saved",font=("Helvetica", 11))],
          [sg.Text("          "),sg.InputText()],
          [sg.Text("          "),sg.Button('Save As CSV'), sg.Button('Save As Excel')],
          [sg.Text("          "),sg.Text("",size = (50,1),key = "result2",font=("Helvetica", 10))]
         ]

window = sg.Window("Web Scraping",layout)

while True:
    event, values = window.read()
    
    if event in (sg.WIN_CLOSED, 'Exit'):
        break
    if event == sg.WIN_CLOSED or event == 'Exit':
        break
    
    if event == "Scrap Reviews":
        out = scraping(values[0])
        s = "Successfully Collected " + out
        window["result1"].Update(s)
    
    if event == "Save As CSV":
        filename = values[1] + ".csv"
        output.to_csv(filename)
        f = filename + " saved successfully at the ipynb file location!"
        window["result2"].Update(f)
    
    if event == "Save As Excel":
        filename = values[1] + ".xlsx"
        output.to_excel(filename)
        f = filename + " saved successfully at the ipynb file location!"
        window["result2"].Update(f)
            
window.close()